# Capacitive simulation of qubit decay in charge line

In [1]:
%load_ext autoreload
%autoreload 2

## Rendering the design

In [2]:
from chip_initialization import create_chip_and_gui
design, gui = create_chip_and_gui()

from design import render_qiskit_metal_design
render_qiskit_metal_design(design, gui)

position: [ 0.       -2.491745] : direction:[0. 1.]. position: [0.       2.731745] : direction:[ 0. -1.].
position: [0.       4.461745] : direction:[-0. -1.]. position: [0.       3.068255] : direction:[0. 1.].
position: [ 0.       -4.461745] : direction:[0. 1.]. position: [ 0.       -3.308255] : direction:[ 0. -1.].
position: [-4.37 -2.  ] : direction:[ 1. -0.]. position: [-4.2 -1.8] : direction:[-1.  0.].
position: [-4.37  2.  ] : direction:[ 1. -0.]. position: [-4.2  1.8] : direction:[-1.  0.].
position: [ 0.       -2.491745] : direction:[0. 1.]. position: [0.       2.731745] : direction:[ 0. -1.].
position: [0.       4.461745] : direction:[-0. -1.]. position: [0.       3.068255] : direction:[0. 1.].
position: [ 0.       -4.461745] : direction:[0. 1.]. position: [ 0.       -3.308255] : direction:[ 0. -1.].
position: [-4.37 -2.  ] : direction:[ 1. -0.]. position: [-4.2 -1.8] : direction:[-1.  0.].
position: [-4.37  2.  ] : direction:[ 1. -0.]. position: [-4.2  1.8] : direction:[-1.  0

## Creating the study and optimization targets

In [3]:
import names as n
import mini_studies as ms
import optimization_targets as ot

In [4]:
MINI_STUDY_GROUP = n.NBR_1
MINI_STUDY = ms.get_mini_study_qb_charge_line(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_qb_charge_line(group=MINI_STUDY_GROUP)

## Creating design analysis objects

In [ ]:
import time
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qdesignoptimizer.utils.utils import get_save_path

# Closing Ansys hfss
from qdesignoptimizer.utils.utils import close_ansys
close_ansys()

In [7]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
    )

design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=get_save_path("out/", n.CHIP_NAME),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CHARGE_LINE_DECAY,
)

[INFO|2025-03-17 11:41:04]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 11:41AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:41AM [load_ansys_project]: 	Opened Ansys App
INFO 11:41AM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 11:41AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/harshva/Documents/Ansoft/
	Project:   Project325
INFO 11:41AM [connect_design]: No active design found (or error getting active design).
INFO 11:41AM [connect]: 	 Connected to project "Project325". No design detected
11:41AM 12s WARNING [activate_ansys_design]: The design_name=get_mini_study_qb_charge_line was not in active project.  Desig

## Optimization step

In [ ]:
# number of runs of optimization and number of passes for simulation at each run
group_runs = 14
group_passes_cap = 8

for i in range(group_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

[INFO|2025-03-17 11:41:14]: Updated_design_vars
{
    "cpw_width": "10 um",
    "cpw_gap": "6 um",
    "design_var_width_qubit_1": "400um",
    "design_var_width_qubit_2": "400um",
    "design_var_cl_pos_x_qubit_1": "-2600um",
    "design_var_cl_pos_y_qubit_1": "-1800um",
    "design_var_cl_pos_x_qubit_2": "-2600um",
    "design_var_cl_pos_y_qubit_2": "1800um",
    "design_var_length_resonator_1_capacitance": "20um",
    "design_var_length_resonator_2_capacitance": "20um",
    "design_var_lj_qubit_1": "12.1nH",
    "design_var_lj_qubit_2": "9.1nH",
    "design_var_cj_qubit_1": "0fF",
    "design_var_cj_qubit_2": "0fF",
    "design_var_length_resonator_1": "7500um",
    "design_var_length_resonator_2": "6500um",
    "design_var_length_coupler_1to2": "4000um",
    "design_var_coupl_length_qubit_1_resonator_1": "100um",
    "design_var_coupl_length_qubit_2_resonator_2": "100um",
    "design_var_coupl_length_resonator_1_tee": "400um",
    "design_var_coupl_length_resonator_2_tee": "160um"


position: [ 0.       -2.491745] : direction:[0. 1.]. position: [0.       2.731745] : direction:[ 0. -1.].
position: [0.       4.461745] : direction:[-0. -1.]. position: [0.       3.068255] : direction:[0. 1.].
position: [ 0.       -4.461745] : direction:[0. 1.]. position: [ 0.       -3.308255] : direction:[ 0. -1.].
position: [-4.37 -2.  ] : direction:[ 1. -0.]. position: [-4.2 -1.8] : direction:[-1.  0.].
position: [-4.37  2.  ] : direction:[ 1. -0.]. position: [-4.2  1.8] : direction:[-1.  0.].
position: [ 0.       -2.491745] : direction:[0. 1.]. position: [0.       2.731745] : direction:[ 0. -1.].
position: [0.       4.461745] : direction:[-0. -1.]. position: [0.       3.068255] : direction:[0. 1.].
position: [ 0.       -4.461745] : direction:[0. 1.]. position: [ 0.       -3.308255] : direction:[ 0. -1.].
position: [-4.37 -2.  ] : direction:[ 1. -0.]. position: [-4.2 -1.8] : direction:[-1.  0.].
position: [-4.37  2.  ] : direction:[ 1. -0.]. position: [-4.2  1.8] : direction:[-1.  0

INFO 11:41AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:41AM [load_ansys_project]: 	Opened Ansys App
INFO 11:41AM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 11:41AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/harshva/Documents/Ansoft/
	Project:   Project325
INFO 11:41AM [connect_design]: 	Opened active design
	Design:    get_mini_study_qb_charge_line [Solution type: Eigenmode]
INFO 11:41AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 11:41AM [connect]: 	Connected to project "Project325" and design "get_mini_study_qb_charge_line" 😀 

INFO 11:41AM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 11:41AM [connect_setup]: 	No design setup detected.
WARNING 11:41AM [connect_setup]: 	Creating Q3D default setup.
INFO 11:41AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:41AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ans

position: [ 0.       -2.491745] : direction:[0. 1.]. position: [0.       2.731745] : direction:[ 0. -1.].
position: [0.       4.461745] : direction:[-0. -1.]. position: [0.       3.068255] : direction:[0. 1.].
position: [ 0.       -4.461745] : direction:[0. 1.]. position: [ 0.       -3.308255] : direction:[ 0. -1.].
position: [-4.37 -2.  ] : direction:[ 1. -0.]. position: [-4.18 -1.8 ] : direction:[-1.  0.].
position: [-4.37  2.  ] : direction:[ 1. -0.]. position: [-4.2  1.8] : direction:[-1.  0.].
position: [ 0.       -2.491745] : direction:[0. 1.]. position: [0.       2.731745] : direction:[ 0. -1.].
position: [0.       4.461745] : direction:[-0. -1.]. position: [0.       3.068255] : direction:[0. 1.].
position: [ 0.       -4.461745] : direction:[0. 1.]. position: [ 0.       -3.308255] : direction:[ 0. -1.].
position: [-4.37 -2.  ] : direction:[ 1. -0.]. position: [-4.18 -1.8 ] : direction:[-1.  0.].
position: [-4.37  2.  ] : direction:[ 1. -0.]. position: [-4.2  1.8] : direction:[-1

INFO 11:41AM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 11:41AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 11:41AM [analyze]: Analyzing setup Setup
INFO 11:41AM [get_matrix]: Exporting matrix data to (C:\Users\harshva\AppData\Local\Temp\tmp86kzhi9g.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 11:41AM [get_matrix]: Exporting matrix data to (C:\Users\harshva\AppData\Local\Temp\tmpnv0jkk8o.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 11:41AM [get_matrix]: Exporting matrix data to (C:\Users\harshva\AppData\Local\Temp\tmp4iyfa0vx.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 11:41AM [get_matrix]: Exporting matrix data to (C:\Users\harshva\AppData\Local\Temp\tmp7gcii_fp.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 

## Update parameters

In [ ]:
design_analysis.overwrite_parameters()

## Close

In [ ]:
close_ansys()